# Lesson 2 - Feedforward models

This lesson will present the *dataflow* paradigm and its application to Artificial Neural Networks.
The students will implement from scratch all the operations required to create their first Deep model, and finally use it to solve a regression task.

### Summary

* [Classical visualization of ANNs](#classicalANNs)
* [The *dataflow* paradigm](#dataflow)
* [Operations](#operations)
* [Running a *dataflow* program](#rundataflowprog)
* [Reinterpreting the *backpropagation* algorithm](#dataflowbackprop)
* [Batch processing](#batch)
* [The *Boston housing prices* problem](#bostonhousing)

<a id=classicalANNs></a>
### Classical visualization of ANNs

Our way to think to problems is strongly influenced by the way we visualize them.
Artificial Neural Networks visualization has been strongly influenced by biological neural networks.
The fundamental processing units, the neurons, are usually depicted as small circles plus their connections.
In the picture below, we highlighted a neuron nucleus in gold, its synapsis (i.e. incoming signals channels) in green and its dendrites (i.e. outgoing signals channels) in turquoise.

<img src='figures/neuralnet_classic.png', width=360, height=360></img>

Although simple, this representation system is highly misleading from a mathematical modelling perspective.
In fact, it induces a graph representation in our mind, thinking to neurons' nucleuses as nodes and connections as directed edges.
It perfectly depicts the structure of the ensemble of fundamental units, but does not convey much of the **hierarchy of feature maps** that each layer of neurons applies to the preceding layers' features.

We can think to the ensemble of the states of the neurons in a particular layer as coordinates of points in a vector space (or its natural extension, a **tensor space**).

<img src='figures/neuralnet_classic_hierarchy.png', width=360, height=360></img>

First, the ANN *looks* at a point $x \in X$ using its input layer (the cyan one in figure); the connections between the input layer and the second (hidden) layer then transform this point into a point $\phi(x) \in \Phi$ in what is called a *latent representation* or **feature** (the green one in figure); finally, the connections from the hidden layer to the output layer trasform this feature into a point $y = f(\phi(x))$ (the magenta one in figure) in the desired output space $Y$.

The main change of paradigm in visualization of ANNs is passing from node=neuron/edge=connection graphs to **node=feature map/edge=tensor data structure** graphs.

<a id=dataflow></a>
### The *dataflow* paradigm

[Dataflow](https://en.wikipedia.org/wiki/Dataflow_programming) is a programming model that thinks to programs as computational graphs, graphs which **nodes are operations** and which **edges contain data that are consumed and produced by the operations** they connect.
We will call consumed data *operands* and produced data *results*.

Why is the *dataflow* paradigm so important for ANNs research?
The main reason is that graph representation of a program exposes important computational optimizations:
* **concurrency**, due to the explicit definition of dependencies (edges);
* **distributed computing**, thanks to the operations placement which is possible due to the operations modeling (nodes).

The degree of concurrency enabled by parallel architectures and parallel programming models played a critical role in the Artificial Neural Networks boom of the last ten years.

<img src='figures/google_dl.png', width=480, height=480></img>

Mastering the **dataflow programming model** requires to approach the programming problem in four stages.
The first three stages compose a **building phase** during which the graph is assembled.
The last stage represents the **execution phase**, when the graph is *brought to life* flowing data through it.

The four stages are:
* **design** an analytical model for you problem;
* **identify the transformations** required by the model;
* **assemble these tranformations consistently**, i.e. respecting their hierarchical dependencies;
* **flow data through the model**, feeding suitable input data to the first transformations and then executing other operations as soon as their required operands become available.

To get a grasp on these concepts, we will go step by step through an example.

During a data analysis, we **designed a statistical model**, parametric in $\theta=(b_1, w_1, w_2)$, expressed by the analytical function:

$$\begin{align} f(x, \theta=(b_1, w_1, w_2)) &= h(x, b_1, w_1)w_2 \\ &= \sigma(\nu(x)w_1 + b_1)w_2 \\ &= \sigma((x - \bar{x})w_1 + b_1)w_2 \end{align}$$

This decomposition highlights the dependencies of each stage of computation on preceeding elements.
Thus we can proceed to **identify the required elementary transformations** proceeding backward from the last (higher level) operation, iteratively asking **"Which operands are required to compute this feature?"**:
* the last operation is $f(x, \theta=(b_1, w_1, w_2)) = h(x, b_1, w_1)w_2$, that it is the product between a vector $h$ and a matrix $w_2$; these operands come from distinct operations;
* $w_2$ is a parameter of our model, so it is data that should just be emitted by some *emitting operation* which has no required dependencies; since parameters can be changed, this operation should somehow allow to emit different data depending on the moment it is executed;
* $h = h(x, b_1, w_1)$ is itself the otput of an operation $h(x, b_1, w_1) = \sigma(\nu(x)w_1 + b_1)$, which requires as operand the quantity $s_1 = \nu(x)w_1 + b_1$;
* $s_1$ is the sum of two operands, namely $\nu(x)w_1$ and $b_1$;
* we can apply to $b_1$ the same reasoning that we applied to $w_2$;
* $\nu(x)w_1$ is the vector-matrix product of the two  operands $\nu(x)$ and $w_1$;
* we can apply to $w_1$ the same reasoning that we applied to $w_2$ and $b_1$;
* $\nu(x) = x - \bar{x}$ is obtained as the difference betweend the independent operands $x$ and $\bar(x)$;
* $\bar{x}$ is not a parameter, but an external constant that should be always emitted equal to itself (where a parameter could change);
* finally, $x$ is the actual input to the statistical model, the observation of the reality: it should be emitted by an operation which is be fed by the real world.

This analysis corresponds to the computational graph depicted below.

<img src='figures/dataflow.png', width=480, height=480></img>

We see that we have **input nodes** that contain:
* data that can be fed in by the real world: **placeholder** operation;
* constant information: **constant** operation;
* parametric information, that can be changed as required: **variable** operations.

Then, we have *real* operations in the form of **internal nodes**:
* **sum** node;
* **vector-matrix multiplication** nodes;
* **sigmoid activation** node.

We said before that the intermediate feature spaces represented by ANNs layers actually are vector or tensor spaces.
Due to the inherent structure of vector and tensor spaces, points in these spaces are naturally encoded by multidimensional arrays data structures.
Thus, **operations** (the circles representing the graph nodes) eventually consume and produce data in the form of **tensors** (the squares on the edges).
Usually, the output tensor of the last feature map is not considered as an edge, since it is not consumed by any operation.

To simplify the program management, operation placement and data transfer, operations and their output tensor are usually merged into a single implemented object called `Node`.

<img src='figures/dataflow_implementation.png', width=480, height=480></img>

Such an object should store basically three attributes:
* `inbound_nodes`, a list of the **nodes whose output is required by the current node**, i.e. the nodes that provide operands to it;
* `outbound_nodes`, a list of **nodes that will use the current node's output**, i.e. the nodes to which it provides operands;
* `state`, an array to store **the result of the operation**; the output tensor itself!

The node should also have a method:
* `forward`, an algorithm to consume the inputs (operands) and produce its output (result).

When a `Node` object is created, it should be linked to the `Node`s that contain its required operand.
Thus, the `__init__` method of such a class should update the information about the graph's connectivity, taking a `list` of inbound `Node`s and communicating to them that the currently created `Node` requires their state (i.e. updating their `outbound_nodes` attribute).
Notice that the connectivity information added by the constructor method of a `Node` is local, since it regards only the edged which have the current `Node` as an extremum.

To visualize this, the next figure highlights a subgraph realizing the feature map $s_1 = l_1(x) = \nu(x)w_1 + b_1$; this subgraph is composed by:
* an arbitrary node as the current node (magenta);
* its `inbound_nodes` (orange), containing the operands $\nu(x)$, $w_1$ and $b_1$;
* its `outbound_nodes` (sea green), where the output tensor $s_1$ is directed.

<img src='figures/dataflow_implementation_Node.png', width=480, height=480></img>

In [14]:
import numpy as np

In [15]:
class Node():
    def __init__(self, inbound_nodes=list()):
        self.inbound_nodes = inbound_nodes
        for node in self.inbound_nodes:
            node.outbound_nodes.append(self)
        self.outbound_nodes = list()
        self.state = None
    
    def forward(self):
        pass
    

<a id=operations></a>
### Operations

#### Input operations
The basic operations of a *dataflow* graph are input operations.
These operations provide *raw data* to the program, data that is not preprocessed or transformed before being fed to the graph.

This information comes in three form:
* `Constant`, an operation which emits an operand always equal to itself; the value of the operand should be communicated to the operation at its creation;
* `Placeholder`, an operation which emits an operand that has been read from the external environment; its `forward` method should thus use a parameter `value` which reads the current observation from the environment;
* `Variable`, an operation which emits an operand that must be initialized by the operation's constructor method; this value is stored as the operation's state; the difference from `Constant` operations will be described in the paragraph about [backpropagation](#dataflowbackprop).

In [19]:
class Constant(Node):
    def __init__(self, value):
        Node.__init__(self)
        self.state = value
        
    def forward(self):
        pass

    
class Placeholder(Node):
    def __init__(self):
        Node.__init__(self)
        
    def forward(self, value=None):
        if value is not None:
            self.state = value
            
            
class Variable(Node):
    def __init__(self, initial_value):
        Node.__init__(self)
        self.state = initial_value
        
    def forward(self):
        pass
    

#### Linear maps and affine transformations
**Linear maps** are the most important transformations that can be applied to vector and tensor spaces.
Let $X$ and $S$ be two vector spaces, that we will call the *input space* and the *score space* respectively.
When both these spaces are finite-dimensional, with dimensions $n$ and $m$ respectively, a linear map

$$l: X \to S$$

can be algebraically described by a **vector-matrix multiplication**

$$\begin{gather} xW = \begin{pmatrix} x_1 & x_2 & \dots & x_n \end{pmatrix}
                      \begin{pmatrix} w_{11} & w_{12} & \dots & w_{1m} \\
                                      w_{21} & w_{22} & \dots & w_{2m} \\
                                      \vdots & \vdots & \ddots & \vdots \\
                                      w_{n1} & w_{n2} & \dots & w_{nm} \end{pmatrix}
\end{gather}$$

The result of this operation is a vector $s \in S$ which components are called *scores*

$$s_k = \sum_{i=1}^{n}x_i w_{ik}, k = 1, \dots m$$

The geometric meaning of this transformation is clear: the $k$-th component of $s$ is the result of a projection operation of $x$ onto the one-dimensional subspace of $X$ idetified by the $k$-th column of $W$.

$$s_k = \langle x, W^{(k)} \rangle, k = 1, \dots m$$

Linear maps are constrained to rotations and homothecies around the origin ([polar decomposition](https://en.wikipedia.org/wiki/Polar_decomposition)), and cannot account for translations.
To model such dependencies, a translation is usually added under the form of a **bias vector** $b \in S$:

$$\begin{align} l: \, &X \to S \\ &x \mapsto xW + b \end{align}$$

In [17]:
class Linear(Node):
    def __init__(self, x, w):
        Node.__init__(self, inbound_nodes=[x, w])
        
    def forward(self):
        x = self.inbound_nodes[0].state
        w = self.inbound_nodes[1].state
        self.state = np.dot(x, w)
        
        
class Add(Node):
    def __init__(self, s, b):
        Node.__init__(self, inbound_nodes=[s, b])
        
    def forward(self):
        s = self.inbound_nodes[0].state
        b = self.inbound_nodes[1].state
        self.state = s + b
        

#### Non-linear activations
As stated by the Universal Approximation Theorem, the power of ANNs emerges when they are able to **distort space to extract non-linear features**.

In order to achieve this, the output scores of linear transformations $s_k, k=1, \dots m$ should be filtered by **sigmoid** activations

$$\begin{align} y_k &= \sigma(s_k) \\ &= \frac{e^{s_k}}{e^{s_k} + 1}\end{align}$$

In practice, many non-linear functions can play the same role.
For example, the **hyperbolic tangent**

$$\begin{align} y_k &= \tanh(s_k) \\ &= 2\sigma(s_k) - 1 \end{align}$$

or the Leaky Rectified Linear Unit (**LeakyReLU**)

$$LeakyReLU_{q}(s_k) = \begin{cases} qs_k, & \mbox{if } s_k \leq 0 \\ s_k, & \mbox{if } s_k > 0 \end{cases}$$

where $q \geq 0$ is a design constant (the case $q = 0$ yields the widespreadly used **ReLU** function).

Why do they exist so many activation functions?
Should not sigmoid suffice due to the UAT?
Although in theory the LeakyReLU and the ReLU functions do not satisfy the hypothesis of the UAT, in real instances they do, and due to their minimal computational complexity with regard to sigmoids and hyperbolic tangents are thus largely used in real models.
Nevertheless, sigmoid and hyperbolic tangents are used since they allow more straightforward statistical interpretations:
* since $\sigma(s_k) \in (0, 1)$, sigmoid units can smoothly approximate boolean behaviour (e.g. the probability that a certain feature has been detected or not);
* since $\tanh(s_k) \in (-1, 1)$ carries sign information, hyperbolic tangent units can test wether a certain feature is inhibitory or excitatory.

In [18]:
class Sigmoid(Node):
    def __init__(self, x):
        Node.__init__(self, inbound_nodes=[x])
        
    def forward(self):
        x = self.inbound_nodes[0]
        e_x = np.exp(x)
        self.state = e_x / (1+e_x)
        

class Tanh(Node):
    def __init__(self, x):
        Node.__init__(self, inbound_nodes=[x])
        
    def forward(self):
        x = self.inbound_nodes[0]
        e_2x = np.exp(2*x)
        self.state = (e_2x - 1.0) / (e_2x + 1.0)
        
        
class LeakyReLU(Node):
    def __init__(self, q, x):
        Node.__init__(self, inbound_nodes=[x])
        self.q = q
        
    def forward(self):
        x = self.inbound_nodes[0].state
        mask = x > 0
        mask = mask + q * (1-mask)
        self.state = x * mask
        
        
class ReLU(LeakyReLU):
    def __init__(self, x):
        LeakyReLU.__init__(self, 0, x)
        

<a id=rundataflowprog></a>
### Running a *dataflow* program

Up to now, we have developed all the components required to create a computational graph.
But before we can run it, we have to accomplish the third stage, **assemble these operations consistently**.
We can tackle this stage dividing it into two tasks:
* assemble a graph;
* automatically detect the computational dependencies between the operations in the graph.

As for the assembly, opearations should be chained manually by the programmer.
Since he is also the creator of the model, he knows which are the higher level features and which operands (i.e. which lower level abstractions) they require.
For example consider a linear unit without non-linear activation as shown in the previous lesson.

<img src='figures/linear_unit_revisited.png', width=480, height=480></img>

In [6]:
class LinearUnitGraph():
    def __init__(self):
        self._build()
        
    def _build(self):
        self.X = Placeholder()
        self.W = Variable(np.random.randn(3, 1))
        self.B = Variable(np.zeros((1, 1)))
        self.s = Add(Linear(self.X, self.W), self.B)

Now we have an assembled graph.
But the `LinearUnitGraph` class still does not contain information on the order of execution of the operations.
Consequently, it can not process automatically the calls to the `forward` methods of each node, because it might call them in an inconsistent order!

To avoid calling `forward` methods by hand (which is also error prone), we can develop an algorithm to solve this problem.

Remember that the constructor methods of the `Node`s take care of storing **local connectivity information**.
Since all `Node` objects in a graph store local connectivity information, we can use it to reconstruct the **global topology** of the graph and to extract dependencies.
Suppose the computational graph is composed by nodes $O_1, O_2, \dots O_p$.
We would like a permutation $O_{i_1}, O_{i_2}, \dots O_{i_p}$ such that all `inbound_nodes` of node $O_{i_{\bar{j}}}$ have indices $i_j$ such that $j < \bar{j}$.
This problem is known as **topological sorting**.

There are many ways to compute the topological sorting of a graph given its connectivity information.
We use [Kahn's algorithm](https://en.wikipedia.org/wiki/Topological_sorting#Algorithms) for our implementation.

In [20]:
def get_connectivity(input_nodes):
    """Create a description of the connections of each node in the graph.

    Recurrent connections (i.e. connections of a node with itself) are excluded.

    Args:
        input_nodes (:obj:`list` of :obj:`Node`): the input operations of
            the model.

    Returns:
        graph (:obj:`dict` of :obj:`dict` of :obj:`set`): a description of the
            graph's connectivity in terms of inbound-outbound nodes of each
            node.

    """

    graph = dict()
    nodes = input_nodes.copy()
    while len(nodes) != 0:
        # select a node
        current_node = nodes.pop(0)
        # if no information has been collected yet, set up dict entry
        if current_node not in graph:
            graph[current_node] = {'inbound': set(), 'outbound': set()}
        # scroll through current node's outbound nodes
        for node in current_node.outbound_nodes:
            # skip recurrent connections (for RNN cells)
            if node == current_node:
                continue
            # if no information has been collected yet, set up dict entry
            if node not in graph:
                nodes.append(node)
                graph[node] = {'inbound': set(), 'outbound': set()}
            # add reciprocal connectivity information
            graph[current_node]['outbound'].add(node)
            graph[node]['inbound'].add(current_node)

    return graph


def topological_sort(input_nodes, graph):
    """Get a consistent sequence of operations on the given graph.

    Args:
        input_nodes (:obj:`list` of :obj:`Node`): the input operations of
            the model.
        graph (:obj:`dict` of :obj:`dict` of :obj:`set`): a description of the
            graph's connectivity.

    Returns:
        sorted_nodes (:obj:`list` of :obj:`Node`): a sequence of operations
            that ensures computational consistency of the model.

    """

    sorted_nodes = list()
    unlocked_nodes = input_nodes.copy()
    while len(unlocked_nodes) != 0:
        # select an inbound-free node and add it the sorted list
        # (it is ok for computation since all "requirement" nodes are available)
        current_node = unlocked_nodes.pop(0)
        sorted_nodes.append(current_node)
        current_outbound = graph[current_node]['outbound']
        if current_outbound is None:
            # dead end reached
            continue
        for node in graph[current_node]['outbound']:
            # free the outbound node from requiring current node
            graph[node]['inbound'].remove(current_node)
            # if the outbound node has no more requirements to be fulfilled,
            # unlock it
            if len(graph[node]['inbound']) == 0:
                unlocked_nodes.append(node)

    return sorted_nodes


def get_graph_flow(input_nodes):
    """Build the network graph.

    A wrapper function to automate model build.

     Args:
        input_nodes (:obj:`list` of :obj:`Node`): the input operations of
            the graph.

     Returns:
        requirements_chain (:obj:`list` of :obj:`Node`): a sequence of
            operations that ensures computational consistency of the model.

   """

    connectivity = get_connectivity(input_nodes)
    requirements_chain = topological_sort(input_nodes, connectivity)

    return requirements_chain


A brief explanation of the functions defined in the previous cell:
* `get_connectivity` takes a list of input nodes and returns a `dictionary` storing an entry for each `Node` summarizing the connectivity of that `Node`;
* `topological_sort` consumes this `dictionary` to produce a `list` of `Node`s starting from the list of input nodes and removing them from their *requirements role for following nodes* (line `69` of previous cell); once a `Node` is freed from all its requirements, it is marked as *unlocked* and add to the list of computable nodes;
* `get_graph_flow` is simply a wrapper for the previous two functions.

If we extend the `LinearUnitGraph` class with a method `_get_topological_order`, we can verify that the algorithm works as desired on this computational graph.

In [21]:
class LinearUnitOrder(LinearUnitGraph):
    def __init__(self):
        LinearUnitGraph.__init__(self)
        self.graph = self._get_topological_order()
        
    def _get_topological_order(self):
        return get_graph_flow([self.X, self.W, self.B])
        
        
linearunitorder = LinearUnitOrder()
for node in linearunitorder.graph:
    print(node)


We can finally **bring the graph to life by flowing data through it**.

We first load the observed data into the `Placeholder` operation of the graph, then execute a *systolic* sequence of calls to the `forward` methods to all the `Node`s until the final `Node`'s `state` is computed.

In [22]:
def forward_prop(requirements_chain):
    """Push the current inputs through the whole model.

    Consistently complete the systolic sequence of operations.

    Args:
        requirements_chain (:obj:`list` of :obj:`Node`): a sequence of
            operations that ensures computational consistency of the model.

    """
    for node in requirements_chain:
        node.forward()


class LinearUnit(LinearUnitOrder):
    def __init__(self):
        LinearUnitOrder.__init__(self)
        
    def infer(self, x):
        self.X.forward(value=x)
        forward_prop(self.graph)

      
linearunit = LinearUnit()

x = np.random.randn(1, 3)
linearunit.infer(x)
print('Observation: ', x)
print('Prediction: ', linearunit.s.state)


Observation:  [[ 0.40074923  0.80374973 -1.76938737]]
Prediction:  [[ 2.39164155]]


<a id=dataflowbackprop></a>
### Reinterpreting the *backpropagation* algorithm

<a id=batch></a>
### Batch processing

<a id=bostonhousing></a>
### The *Boston housing prices* problem

### References